In [1]:
import os
import pandas as pd
#import psycopg2 as pg2
import warnings
import pyodbc

warnings.filterwarnings("ignore")

In [2]:
import json

with open('config.json') as f:
    config = json.load(f)

server = config['server']
database = config['database']

conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = conn.cursor()

In [3]:
cursor.execute("""
    INSERT INTO olympics.OlympicGames (GameName, Year, Season, HostCity)
SELECT 
    game,
    year,
    season,
    host_city
FROM olympics.host_cities;
"""
)


In [4]:
conn.commit()

In [5]:
cursor.execute("""
    INSERT INTO olympics.Sport(Sportname, GameName)
SELECT DISTINCT 
    sport,
    game
FROM olympics.athlete_data
WHERE game IN (SELECT GameName FROM olympics.OlympicGames);
"""
)


In [6]:
conn.commit()

In [7]:
cursor.execute("""
    INSERT INTO olympics.Event (Eventname, Sportname, GameName)
SELECT DISTINCT 
    event,
    sport,
    game
FROM olympics.athlete_data
WHERE sport IN (SELECT SportName FROM olympics.Sport)
  AND game IN (SELECT GameName FROM olympics.Sport);

"""
)


In [8]:
conn.commit()

In [12]:
cursor.execute("""
    INSERT INTO olympics.noc_Country (noc, country)
SELECT DISTINCT 
    noc,
    country
FROM olympics.countries_noc
"""
)

In [13]:
conn.commit()

In [16]:
cursor.execute("""
    INSERT INTO olympics.Athlete (AthleteId, AthleteName, Gender, Born, Died, BirthYear, DeathYear, Weight, Height, Team)
SELECT DISTINCT 
    id,
    name,
    gender,
    born,
    died,
    birth_year,
    death_year,
    weight,
    height,
    team
FROM olympics.athlete_data
WHERE team IN (SELECT country FROM olympics.countries_noc)
"""
)

In [17]:
conn.commit()

In [21]:
cursor.execute("""
    INSERT INTO olympics.Participation (AthleteId, GameName, SportName, EventName, Position, Medal)
SELECT DISTINCT 
    id,
    game,
    sport,
    event,
    position,
    medal
FROM olympics.athlete_data
WHERE sport IN (SELECT SportName FROM olympics.Event)
  AND game IN (SELECT GameName FROM olympics.Event)
  AND event IN (SELECT EventName FROM olympics.Event)
  AND id IN (SELECT AthleteId FROM olympics.Athlete);
"""
)

In [22]:
conn.commit()